In [6]:
# imports
import io
import base64
import pathlib
import datetime
import random
import numpy as np
import json
import torch
import torch.nn as nn

from matplotlib import pyplot as plt
%matplotlib inline
from tqdm import tqdm
from model import NCA_model

from IPython.display import Image, clear_output
import PIL.Image, PIL.ImageDraw

In [7]:
# utility functions

# creates a circle mask given a size, radius and position
def create_circle_mask(size, radius, pos):
    pos = pos * size
    Y, X = np.ogrid[:size, :size]
    dist_from_center = np.sqrt((X - pos[0])**2 + (Y-pos[1])**2)
    mask = dist_from_center >= radius
    return mask

# Loads an image from a specified path and converts to torch.Tensor
def load_image(path, size):
    img = PIL.Image.open(path)
    img = img.resize((size, size), PIL.Image.Resampling.BILINEAR)
    img = np.float32(img) / 255.0
    img[..., :3] *= img[..., 3:]
    return torch.from_numpy(img).permute(2, 0, 1)[None, ...]

# converts an RGBA image to a RGB image
def to_rgb(img_rgba):
    rgb, a = img_rgba[:, :3, ...], torch.clamp(img_rgba[:, 3:, ...], 0, 1)
    return torch.clamp(1.0 - a + rgb, 0, 1)

def np2pil(a):
    if a.dtype in [np.float32, np.float64]:
        a = np.uint8(np.clip(a, 0, 1)*255)
    return PIL.Image.fromarray(a)

def imwrite(f, a, fmt=None):
    a = np.asarray(a)
    if isinstance(f, str):
        fmt = f.rsplit('.', 1)[-1].lower()
    if fmt == 'jpg':
        fmt = 'jpeg'
    file = open(f, 'wb')
    np2pil(a).save(file, fmt, quality=95)
  
def imencode(a, fmt='jpeg'):
    a = np.asarray(a)
    if len(a.shape) == 3 and a.shape[-1] == 4:
        fmt = 'png'
    f = io.BytesIO()
    imwrite(f, a, fmt)
    return f.getvalue()

def imshow(a, fmt='jpeg'):
    display(Image(data=imencode(a, fmt)))
  
def zoom(img, scale=4):
    img = np.repeat(img, scale, 0)
    img = np.repeat(img, scale, 1)
    return img
 
def show_batch(batch):
    batch_size = batch.shape[0]
    fig, axs = plt.subplots(1, batch_size)
    fig.suptitle('batch viz')
    plt.axis('off')
    for i in range(batch_size):
        img_i = batch[i]
        img_i = img_i[:4].unsqueeze(0)
        print ('img_i.shape: ', img_i.shape)
        # show_image(img_i)
    #     img_rgb = to_rgb(img).squeeze().permute(1, 2, 0)
    #     axs[i].imshow(img_rgb)
    # plt.show(fig)

# Create a starting tensor for training
# Only the active pixels are goin to be in the middle
def make_seed(size, n_channels):
    x = torch.zeros((1, n_channels, size, size), dtype=torch.float32)
    x[:, 3:, size // 2, size // 2] = 1
    return x

In [8]:
# used for experiments

import sys
np.set_printoptions(threshold=sys.maxsize)
torch.set_printoptions(threshold=sys.maxsize)
torch.set_printoptions(profile="full")

x = torch.rand(2, 3, 3, 3)
#print (x)

z = torch.zeros(1, 3, 3, 1)

x[-1:] *= z
#print (x)

In [9]:
# training parameters
img = 'imgs\pup.png'
name = 'pup40'
save_model = True

size = 40
pad = 16
n_channels = 16

n_train_iter = 5000
batch_size = 8
pool_size = 1024
n_damage = 3

eval_freq = 500
eval_iter = 300

log_dir = 'logs'
model_dir = 'models'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print ('cuda available? ', torch.cuda.is_available())
print ('device: ', device)

cuda available?  True
device:  cuda


In [10]:
# misc
full_size = size + (2 * pad)
device = torch.device(device)

# create log
log_path = pathlib.Path(log_dir)
log_path.mkdir(parents=True, exist_ok=True)

# target image
loaded_img = load_image(img, size)
target_img_ = nn.functional.pad(loaded_img, (pad, pad, pad, pad), 'constant', 0)
target_img = target_img_.to(device)
target_img = target_img.repeat(batch_size, 1, 1, 1)



img_rgb = to_rgb(loaded_img).squeeze().permute(1, 2, 0)
print ('img_rgb.shape: ', img_rgb.shape)

print ('target image: ')
imshow(zoom(img_rgb), fmt='png')

img_rgb.shape:  torch.Size([40, 40, 3])
target image: 


AttributeError: '_io.BytesIO' object has no attribute 'name'

In [ ]:
# model and optimizer
model = NCA_model(_n_channels=n_channels, _device=device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)

# pool init
seed = make_seed(size, n_channels).to(device)
seed = nn.functional.pad(seed, (pad, pad, pad, pad), 'constant', 0)
pool = seed.clone().repeat(pool_size, 1, 1, 1)

# training loop
for it in tqdm(range(n_train_iter)):
    batch_ixs = np.random.choice(
        pool_size, batch_size, replace=False
    ).tolist()
    
    # get training batch
    x = pool[batch_ixs]
    
    # damage examples in batch
    if n_damage > 0:
        radius = random.uniform(size*0.1, size*0.4)
        u = random.uniform(0, 1) * size + pad
        v = random.uniform(0, 1) * size + pad
        mask = create_circle_mask(full_size, radius, [u, v])
        x[-n_damage:] *= torch.tensor(mask).to(device)
        
    # visualize batch
    show_batch(x)
    
    # forward pass
    for i in range(np.random.randint(64, 96)):
        x = model(x)
    
    loss_batch = ((target_img - x[:, :4, ...]) ** 2).mean(dim=[1, 2, 3])
    loss = loss_batch.mean()
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print ('train/loss: ', it, '/', loss)
    
    # find best in batch
    argmax_batch = loss_batch.argmax().item()
    argmax_pool = batch_ixs[argmax_batch]
    remaining_batch = [i for i in range(batch_size) if i != argmax_batch]
    remaining_pool = [i for i in batch_ixs if i != argmax_pool]
    
    pool[argmax_pool] = seed.clone()
    pool[remaining_pool] = x[remaining_batch].detach()
    clear_output(wait=True)
    
    if it % eval_freq == 0:
        x_eval = seed.clone()
        eval_video = torch.empty(1, eval_freq, 3, *x_eval.shape[2:])
        for it_eval in range(eval_iter):
            x_eval = model(x_eval)
            x_eval_out = to_rgb(x_eval[:, :4].detach().cpu())
            eval_video[0, it_eval] = x_eval_out
                    
# save model
if save_model:
    model_path = pathlib.Path(model_dir)
    model_path.mkdir(parents=True, exist_ok=True)
    if name == None:
        ts = str(datetime.datetime.now()).replace(' ', '_').replace(':', '-').replace('.', '-')
        name = 'model_' + ts
    torch.save(model, model_dir + '\\' + name + '.pt')
    
    # save model parameters
    dict = {
        "img": img,
        "batch_size": batch_size,
        "device": device,
        "eval_freq": eval_freq,
        "eval_iter": eval_iter,
        "n_train_iter": n_train_iter,
        "n_channels": n_channels,
        "log_dir": log_dir,
        "pad": pad,
        "pool_size": pool_size,
        "size": size,
        "save_model": save_model,
        "model_dir": model_dir,
        "name": name,
        "damage": n_damage
    }
    json_object = json.dumps(dict, indent=4)
    with open(model_dir + '\\' + name + '_params.json', 'w') as outfile:
        outfile.write(json_object)